# LangGraph

In diesem Notebook werden wir uns mit LangGraph beschäftigen, einer Erweiterung des LangChain-Frameworks, die darauf ausgerichtet ist, komplexe, zustandsbehaftete Workflows mit LLMs zu modellieren.

## Lernziele

- LangGraph's Architektur und Grundkonzepte verstehen
- Den Unterschied zwischen LangChain und LangGraph erkennen
- Einfache Agenten mit LangGraph erstellen
- Komplexere Agenten mit mehreren Tools implementieren
- LangGraph in bestehende Systeme integrieren

## Setup und Imports

In [ ]:
# Benötigte Pakete importieren
from typing import Annotated, Dict, List, Tuple, TypedDict, Union, Any
import operator
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langgraph.graph import StateGraph, END
from langgraph.graph import MessageGraph

load_dotenv()  # Laden der Umgebungsvariablen aus .env

In [ ]:
# LLM initialisieren
def llm(temperature=0.0, model_name="gpt-4o"):
    return ChatOpenAI(temperature=temperature, model_name=model_name)

## 1. Was ist LangGraph?

LangGraph ist ein Framework, das auf LangChain aufbaut und speziell für die Erstellung zustandsbehafteter, multi-actor Anwendungen mit LLMs konzipiert wurde. Es stellt einen Übergang von den einfachen sequentiellen Ketten in LangChain zu komplexen Graphen mit Verzweigungen, Schleifen und mehreren Akteuren dar.

### Hauptmerkmale von LangGraph:

- **Zustandsverwaltung**: Explizite Verwaltung von Zuständen während des Ausführungsflusses
- **Gerichtete Graphen**: Anwendungslogik wird als gerichteter Graph modelliert
- **Verzweigungen und Konditionen**: Möglichkeit, basierend auf Bedingungen verschiedene Pfade einzuschlagen
- **Typisierung**: Starke Typisierung für bessere Entwicklererfahrung und Fehlererkennung
- **Persistenz**: Optional zustandsbehaftete Ausführung über mehrere Anfragen hinweg

## 2. LangChain vs. LangGraph

### LangChain:
- Fokus auf sequentielle Verarbeitung (Chains)
- Komponentenorientiert (LLMs, Embeddings, Vektorspeicher, etc.)
- Gut für einfache Pipelines wie RAG-Anwendungen

### LangGraph:
- Fokus auf komplexe zustandsbehaftete Workflows
- Graphenbasierte Struktur mit bedingter Logik
- Unterstützt Zyklen/Schleifen für iterative Prozesse
- Ideal für Agenten, die mehrere Schritte und Entscheidungen erfordern

**LangGraph ergänzt LangChain, ersetzt es aber nicht.** Beide Frameworks können und sollten zusammen verwendet werden.

## 3. Einfaches Beispiel: Ein simpler Chatbot mit LangGraph

Beginnen wir mit einem einfachen Beispiel: Ein Chatbot, der auf Nachrichten reagiert.

In [ ]:
# Einfachen MessageGraph erstellen
model = llm()

# Graph bauen
graph_builder = MessageGraph()

# Wir definieren unser LLM als einzigen Knoten
graph_builder.add_node("chatbot_node", model)

# Dieser Knoten wird der Entrypoint
graph_builder.set_entry_point("chatbot_node")

# Von diesem Knoten geht es direkt zu "END", dem vordefinierten Terminalknoten
graph_builder.set_finish_point("chatbot_node")

# Jetzt kompilieren wir
simple_graph = graph_builder.compile()

In [ ]:
# Testen des Chatbots
messages = [
    HumanMessage(content="Hallo! Wie geht es dir?")
]

response = simple_graph.invoke(messages)
print(response[0].content)

## 4. Zustandsbehafteter Graph mit typisierten Zuständen

Jetzt implementieren wir einen zustandsbehafteten Graphen mit expliziter Typisierung.

In [ ]:
# Definieren des Zustands mit Typisierung
class ChatState(TypedDict):
    """Repräsentiert den Zustand unseres Chat-Agenten."""
    messages: Annotated[List[BaseMessage], operator.add]  # Liste von Nachrichten, die mit operator.add kombiniert werden können
    count: int  # Zähler für die Anzahl der Interaktionen

In [ ]:
# Definieren von zwei Knoten für unseren Graphen

# 1. LLM Node - generiert Antworten
def call_llm(state: ChatState) -> ChatState:
    """Ruft das LLM auf und gibt eine Antwort zurück."""
    # Fügt Systemanweisungen hinzu, wenn es die erste Nachricht ist
    if state["count"] == 0:
        messages = [SystemMessage(content="Du bist ein hilfsbereicher KI-Assistent. Halte Antworten kurz und präzise.")]
        messages.extend(state["messages"])
    else:
        messages = state["messages"]
    
    # LLM aufrufen
    response = model.invoke(messages)
    
    # Neuen Zustand zurückgeben
    return {"messages": [response], "count": state["count"] + 1}

# 2. Counter Node - zählt die Anzahl der Nachrichten
def check_count(state: ChatState) -> str:
    """Überprüft den Zähler und entscheidet, ob das Gespräch fortgesetzt werden soll."""
    if state["count"] >= 5:
        # Nach 5 Nachrichten beenden wir die Konversation
        return "end"
    else:
        return "continue"

In [ ]:
# Graph erstellen
chat_graph_builder = StateGraph(ChatState)

# Knoten hinzufügen
chat_graph_builder.add_node("llm", call_llm)

# Startpunkt definieren
chat_graph_builder.set_entry_point("llm")

# Bedingte Kanten hinzufügen
chat_graph_builder.add_conditional_edges(
    "llm",
    check_count,
    {
        "continue": "llm",  # Schleife zurück zum LLM
        "end": END  # Ende der Konversation
    }
)

# Graph kompilieren
chat_graph = chat_graph_builder.compile()

In [ ]:
# Testen des zustandsbehafteten Chatbots
initial_state = {"messages": [HumanMessage(content="Erzähle mir etwas über KI.")], "count": 0}

# Wir verwenden stream, um die Antworten live zu sehen
for event in chat_graph.stream(initial_state):
    if 'messages' in event:
        for message in event['messages']:
            if isinstance(message, AIMessage):
                print(f"Runde {event['count']-1}: {message.content}\n")

## 5. Agent mit Tool-Verwendung

Jetzt erstellen wir einen komplexeren Agenten, der Tools verwenden kann.

In [ ]:
from langchain.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
import json
import datetime
import math

# Tool 1: Aktuelle Zeit anzeigen
@tool
def get_current_time() -> str:
    """Gibt die aktuelle Zeit und das aktuelle Datum zurück."""
    now = datetime.datetime.now()
    return f"Aktuelles Datum und Uhrzeit: {now.strftime('%d.%m.%Y %H:%M:%S')}"

# Tool 2: Mathematische Berechnungen durchführen
@tool
def calculate(expression: str) -> str:
    """Berechnet den Wert eines mathematischen Ausdrucks. 
    Unterstützt grundlegende Operationen wie +, -, *, /, sowie math-Funktionen.
    
    Args:
        expression: Mathematischer Ausdruck als String, z.B. "2 * 3 + 4" oder "math.sin(0.5)"
    """
    try:
        # Sicherer Namespace mit nur bestimmten math-Funktionen
        safe_dict = {
            'abs': abs, 'round': round,
            'math': math
        }
        
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return f"Das Ergebnis von {expression} ist {result}"
    except Exception as e:
        return f"Fehler bei der Berechnung: {str(e)}"

# Tool 3: Einfache Datensuche (simuliert)
@tool
def search_database(query: str) -> str:
    """Sucht in einer simulierten Datenbank nach Informationen.
    
    Args:
        query: Suchbegriff oder -phrase
    """
    # Simulierte Datenbankeinträge
    database = {
        "python": "Python ist eine interpretierte Hochsprache, die einfach zu erlernen ist und vielseitig eingesetzt wird.",
        "langchain": "LangChain ist ein Framework zur Entwicklung von Anwendungen mit Sprachmodellen.",
        "langgraph": "LangGraph ist eine Erweiterung von LangChain für komplexe zustandsbehaftete Workflows.",
        "llm": "LLM steht für Large Language Model, ein Sprachmodell, das auf großen Textmengen trainiert wurde."
    }
    
    # Einfache Suche nach Schlüsselwörtern
    query = query.lower()
    for key, value in database.items():
        if query in key or key in query:
            return value
    
    return f"Keine Informationen zu '{query}' gefunden."

# Alle Tools in einer Liste sammeln
tools = [get_current_time, calculate, search_database]

In [ ]:
# LLM mit Tool-Unterstützung konfigurieren
tool_model = llm().bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [ ]:
# State Typ für unseren Agenten definieren
class AgentState(TypedDict):
    """State-Typ für unseren Tool-verwendenden Agenten."""
    messages: Annotated[List[BaseMessage], operator.add]
    # Weitere Felder können je nach Bedarf hinzugefügt werden

In [ ]:
# Die beiden Hauptfunktionen für unseren Agenten definieren

# 1. Funktion zum Aufrufen des LLM
def call_model(state: AgentState) -> AgentState:
    """Ruft das LLM auf und entscheidet, ob Tools verwendet werden sollen."""
    messages = state["messages"]
    response = tool_model.invoke(messages)
    return {"messages": [response]}

# 2. Funktion zum Ausführen der Tools
def call_tools(state: AgentState) -> AgentState:
    """Führt Tools basierend auf LLM-Anforderungen aus."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Überprüfen, ob Tool-Calls vorhanden sind
    if not hasattr(last_message, "tool_calls") or not last_message.tool_calls:
        return {"messages": []}
    
    # Tools-Dictionary zum schnellen Nachschlagen erstellen
    tool_dict = {tool.name: tool for tool in tools}
    
    # Neue Nachrichten für die Tool-Ergebnisse
    new_messages = []
    
    # Jede Tool-Anfrage ausführen
    for tool_call in last_message.tool_calls:
        tool_name = tool_call.name
        arguments = json.loads(tool_call.args)
        
        # Überprüfen, ob das Tool existiert
        if tool_name in tool_dict:
            tool_to_call = tool_dict[tool_name]
            
            # Tool mit Argumenten aufrufen
            if arguments:
                tool_result = tool_to_call(**arguments)
            else:
                tool_result = tool_to_call()
                
            # Nachrichten mit Tool-Ergebnis hinzufügen
            new_messages.append(
                AIMessage(
                    content="",
                    tool_call_id=tool_call.id,
                    name=tool_name,
                    tool_calls=[],
                    additional_kwargs={"tool_responses": [tool_result]}
                )
            )
    
    return {"messages": new_messages}

In [ ]:
# Hilfs- und Routing-Funktionen

# Funktion zum Überprüfen, ob weitere Tool-Calls nötig sind
def should_continue(state: AgentState) -> str:
    """Bestimmt, ob der Agent weiter Tools ausführen soll oder fertig ist."""
    messages = state["messages"]
    last_message = messages[-1]
    
    # Wenn das letzte Nachricht keine Tool-Calls hat und vom LLM kommt, beenden
    if isinstance(last_message, AIMessage) and not hasattr(last_message, "tool_calls") or \
       (hasattr(last_message, "tool_calls") and not last_message.tool_calls):
        return "end"
    
    # Sonst: Weiter mit Tool-Ausführung oder Planung
    return "continue"

In [ ]:
# Graph aufbauen
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")

# Bedingungen für Verzweigungen hinzufügen
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

# Graph kompilieren
agent_graph = workflow.compile()

In [ ]:
# Systemanweisung hinzufügen
system_message = SystemMessage(
    content="Du bist ein hilfreicher KI-Assistent. Du kannst verschiedene Tools verwenden, um Aufgaben zu erledigen. "
             "Verwende die Tools, wenn es hilfreich ist, um die Anfrage des Benutzers zu beantworten."
)

# Testen des Agenten mit einer Frage, die Tool-Verwendung erfordert
human_message = HumanMessage(
    content="Kannst du mir sagen, wie spät es ist und dann die Quadratwurzel von 144 berechnen?"
)

# Initial State mit System- und Human-Message
initial_state = {"messages": [system_message, human_message]}

# Agent aufrufen und Ergebnisse anzeigen
result = agent_graph.invoke(initial_state)

# Alle Nachrichten im Dialog ausgeben
for message in initial_state["messages"] + result["messages"]:
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, SystemMessage):
        print(f"System: {message.content}")
    elif isinstance(message, AIMessage):
        if hasattr(message, "tool_call_id") and message.tool_call_id:
            # Tool-Antworten anzeigen
            tool_responses = message.additional_kwargs.get("tool_responses", [])
            if tool_responses:
                print(f"Tool ({message.name}): {tool_responses[0]}")
        elif hasattr(message, "tool_calls") and message.tool_calls:
            # Tool-Anfragen anzeigen
            print(f"AI (thinking): Ich sollte Tools verwenden...")
            for tool_call in message.tool_calls:
                args = json.loads(tool_call.args)
                args_str = ", ".join([f"{k}='{v}'" for k, v in args.items()])
                print(f"AI (tool request): {tool_call.name}({args_str})")
        else:
            # Normale Nachricht anzeigen
            print(f"AI: {message.content}")

## 6. Weitere Beispiele für Tool-Verwendung

In [ ]:
# Weitere Anfragen an unseren Agenten stellen
questions = [
    "Was ist LangChain?",
    "Berechne den Sinus von 30 Grad.",
    "Was ist LangGraph und wie unterscheidet es sich von LangChain?"
]

for question in questions:
    print(f"\n\n=== Neue Anfrage: {question} ===")
    human_message = HumanMessage(content=question)
    initial_state = {"messages": [system_message, human_message]}
    
    result = agent_graph.invoke(initial_state)
    
    # Alle Nachrichten im Dialog ausgeben
    for message in initial_state["messages"] + result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        elif isinstance(message, SystemMessage):
            pass  # System-Nachrichten nicht jedes Mal anzeigen
        elif isinstance(message, AIMessage):
            if hasattr(message, "tool_call_id") and message.tool_call_id:
                # Tool-Antworten anzeigen
                tool_responses = message.additional_kwargs.get("tool_responses", [])
                if tool_responses:
                    print(f"Tool ({message.name}): {tool_responses[0]}")
            elif hasattr(message, "tool_calls") and message.tool_calls:
                # Tool-Anfragen anzeigen
                print(f"AI (thinking): Ich sollte Tools verwenden...")
                for tool_call in message.tool_calls:
                    args = json.loads(tool_call.args)
                    args_str = ", ".join([f"{k}='{v}'" for k, v in args.items()])
                    print(f"AI (tool request): {tool_call.name}({args_str})")
            else:
                # Normale Nachricht anzeigen
                print(f"AI: {message.content}")

## 7. Integration von LangGraph in bestehende Systeme

LangGraph kann sehr gut in bestehende Systeme integriert werden. Hier sind einige wichtige Aspekte:

In [ ]:
# Beispiel für persisted graph (Zustandspersistenz über mehrere Anfragen)
from langgraph.graph.graph import MemorylessGraph
import uuid

# Wir erstellen den gleichen Graphen wie oben, aber für persistente Verwendung
persisted_graph = agent_graph.with_config(
    channel_factories={"thread": lambda: uuid.uuid4().hex}  # Thread-ID für persistenten Zustand
)

# Verwendung mit persist=True, um Zustand zu speichern
thread_id = persisted_graph.get_channel("thread")
print(f"Thread ID: {thread_id}")

# Erste Nachricht
first_question = "Welcher Tag ist heute und welches Jahr haben wir?"
initial_state = {"messages": [system_message, HumanMessage(content=first_question)]}
response = persisted_graph.invoke(initial_state, config={"configurable": {"thread": thread_id}})

print("=== Erste Anfrage ===")
for message in response["messages"]:
    if isinstance(message, AIMessage):
        if hasattr(message, "tool_call_id") and message.tool_call_id:
            tool_responses = message.additional_kwargs.get("tool_responses", [])
            if tool_responses:
                print(f"Tool ({message.name}): {tool_responses[0]}")
        elif hasattr(message, "tool_calls") and message.tool_calls:
            pass  # Tool-Anfragen nicht anzeigen für Übersichtlichkeit
        else:
            print(f"AI: {message.content}")

In [ ]:
# Zweite Nachricht mit Bezug zur ersten
second_question = "Wie viele Tage sind es bis zum Ende des Jahres?"
follow_up_state = {"messages": [HumanMessage(content=second_question)]}

# Den gleichen thread_id verwenden, um den Zustand beizubehalten
response = persisted_graph.invoke(follow_up_state, config={"configurable": {"thread": thread_id}})

print("\n=== Zweite Anfrage (mit Bezug zur ersten) ===")
for message in response["messages"]:
    if isinstance(message, AIMessage):
        if hasattr(message, "tool_call_id") and message.tool_call_id:
            tool_responses = message.additional_kwargs.get("tool_responses", [])
            if tool_responses:
                print(f"Tool ({message.name}): {tool_responses[0]}")
        elif hasattr(message, "tool_calls") and message.tool_calls:
            pass  # Tool-Anfragen nicht anzeigen für Übersichtlichkeit
        else:
            print(f"AI: {message.content}")

## 8. Praktische Anwendungsfälle für LangGraph

LangGraph eignet sich besonders gut für:

1. **Multi-Agent-Systeme**: Verschiedene Agenten arbeiten zusammen, jeder mit eigener Expertise
2. **Komplexe Entscheidungsbäume**: Anwendungen, die mehrere Entscheidungen treffen müssen
3. **Iterative Prozesse**: Aufgaben, die mehrere Zyklen von Planung, Ausführung und Bewertung erfordern
4. **Geschäftsprozessautomatisierung**: Abbildung von realen Workflows mit Entscheidungspunkten
5. **Konversationsagenten mit Gedächtnis**: Chatbots, die Kontext über mehrere Sitzungen beibehalten

Praktische Beispiele:
- Kundenservice-Agenten, die Anfragen kategorisieren und an spezialisierte Sub-Agenten weiterleiten
- Forschungsassistenten, die iterativ Informationen sammeln, bewerten und strukturieren
- Workflow-Automatisierung in Unternehmen mit mehreren Genehmigungsebenen

## 9. Vor- und Nachteile von LangGraph

### Vorteile:
- Explizite Zustandsverwaltung für komplexe Workflows
- Typsicherheit durch TypedDict und Annotationen
- Flexible Verzweigungslogik für bedingte Ausführung
- Unterstützung für zustandsbehaftete Anwendungen
- Nahtlose Integration mit LangChain

### Nachteile:
- Steilere Lernkurve im Vergleich zu einfachen LangChain-Ketten
- Mehr Boilerplate-Code für einfache Anwendungsfälle
- Noch in aktiver Entwicklung, API kann sich ändern
- Komplexere Fehlerbehebung bei größeren Graphen

## 10. Übung: Erweitere den Agenten

Jetzt bist du an der Reihe! Erweitere den Agenten um mindestens eine der folgenden Funktionalitäten:

1. Füge ein neues Tool hinzu (z.B. Wetter-API, Übersetzung, etc.)
2. Implementiere eine bedingte Verzweigung basierend auf dem Nachrichteninhalt
3. Füge einen Logging-Knoten hinzu, der Nachrichten protokolliert
4. Implementiere ein "Human-in-the-loop"-Feedback-System für kritische Entscheidungen

Hier ist ein Gerüst, das du als Ausgangspunkt verwenden kannst:

In [ ]:
# Dein Code hier

# Beispiel für ein zusätzliches Tool:
@tool
def translate(text: str, target_language: str) -> str:
    """Übersetzt einen Text in die Zielsprache (simuliert).
    
    Args:
        text: Der zu übersetzende Text
        target_language: Die Zielsprache (z.B. 'englisch', 'französisch')
    """
    # Hier würde man normalerweise eine echte Übersetzungs-API aufrufen
    # Für Übungszwecke simulieren wir die Antwort
    languages = {
        "englisch": "This is a simulated translation to English.",
        "französisch": "C'est une traduction simulée en français.",
        "spanisch": "Esta es una traducción simulada al español.",
        "italienisch": "Questa è una traduzione simulata in italiano."
    }
    
    target_language = target_language.lower()
    if target_language in languages:
        return f"Übersetzung von '{text}': {languages[target_language]}"
    else:
        return f"Die Sprache '{target_language}' wird nicht unterstützt."

## Zusammenfassung

In diesem Notebook haben wir die Grundlagen von LangGraph kennengelernt und wie es sich von LangChain unterscheidet. Wir haben:

1. Die Architektur und Grundkonzepte von LangGraph kennengelernt
2. Den Unterschied zwischen LangChain und LangGraph verstanden
3. Einfache Chatbots mit MessageGraph erstellt
4. Zustandsbehaftete Graphen mit TypedDict implementiert
5. Komplexere Agenten mit Tool-Unterstützung entwickelt
6. Persistente Zustandsverwaltung über mehrere Anfragen hinweg gesehen
7. Praktische Anwendungsfälle für LangGraph besprochen

LangGraph ist ein leistungsstarkes Framework für die Entwicklung komplexer KI-Anwendungen, die über einfache sequentielle Prozesse hinausgehen. Es ergänzt LangChain optimal und bietet die nötige Flexibilität für anspruchsvolle Anwendungsfälle.